![NERSC Logo](NERSClogo087.png)

# Part 3 Notebook 2: Introduction CUDA Kernel Optimization with Tiling & Shared Memory

Welcome to the Notebook 2 of our CUDA training! In Notebook 1, we learned how to write, compile, and run basic CUDA kernels. We successfully parallelized vector addition, but we used a *naive* approach for matrix multiplication.

**Goal:** Understand *why* our first matrix multiplication kernel was slow and how to fix it using one of the most important CUDA optimization techniques: **tiling with shared memory**.

## Learning Objectives

By the end of this notebook, you will be able to:

* **Identify** Global Memory as a performance bottleneck.
* **Explain** the concept of Shared Memory and its role in data reuse.
* **Implement** a *tiled* matrix multiplication kernel.
* **Use** `__syncthreads()` to manage thread block synchronization.
* **Measure** and **compare** the performance of a naive vs. an optimized kernel.
* **Demonstrate** *why* synchronization is critical by running a "broken" kernel.

## 1. Setup & Baseline: The Global Memory Bottleneck

First, let's set up our environment just like in Part 1. We also need a baseline. The following code is a *naive* matrix multiplication kernel (`C = A * B`).

Every thread computes one element of `C`. To do this, it must read one entire row of `A` and one entire column of `B`. All these reads go directly to **Global Memory (DRAM)**, which is *very slow*.

In [ ]:
%%writefile baseline_matmul.cu

#include <stdio.h>
#include <stdlib.h>

// Helper function to check for CUDA errors
void check(cudaError_t err, const char* msg) {
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA Error: %s: %s\n", msg, cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
}

/* 
 * Naive Kernel: C[row, col] = A[row, :] * B[:, col]
 * Each thread reads N*2 elements from slow Global Memory.
 */
__global__ void matmul_global(float* A, float* B, float* C, int N) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N) {
        float val = 0.0f;
        for (int k = 0; k < N; k++) {
            val += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = val;
    }
}

int main() {
    int N = 1024;
    size_t size = N * N * sizeof(float);
    printf("[Global Kernel] Matrix size: %d x %d\n", N, N);

    // Allocate Host memory
    float* h_A = (float*)malloc(size);
    float* h_B = (float*)malloc(size);
    float* h_C = (float*)malloc(size);
    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f;
        h_B[i] = 2.0f;
    }

    // Allocate Device memory
    float *d_A, *d_B, *d_C;
    check(cudaMalloc(&d_A, size), "cudaMalloc A");
    check(cudaMalloc(&d_B, size), "cudaMalloc B");
    check(cudaMalloc(&d_C, size), "cudaMalloc C");

    // Copy to Device
    check(cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice), "Memcpy A");
    check(cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice), "Memcpy B");

    // --- START: CUDA Event Timing --- 
    cudaEvent_t start, stop;
    check(cudaEventCreate(&start), "EventCreate start");
    check(cudaEventCreate(&stop), "EventCreate stop");
    // --- END: CUDA Event Timing --- 

    // Launch Kernel
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((N + 15) / 16, (N + 15) / 16);

    check(cudaEventRecord(start), "EventRecord start");
    matmul_global<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);
    check(cudaEventRecord(stop), "EventRecord stop");

    // Synchronize host to wait for the kernel to finish
    check(cudaEventSynchronize(stop), "EventSynchronize");

    float time_ms = 0;
    check(cudaEventElapsedTime(&time_ms, start, stop), "EventElapsedTime");
    printf("GLOBAL KERNEL TIME: %.3f ms\n", time_ms);

    // Free memory
    check(cudaEventDestroy(start), "EventDestroy start");
    check(cudaEventDestroy(stop), "EventDestroy stop");
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);

    return 0;
}


In [ ]:
## these modules will load the cudatoolkit on perlmutter
!module load cudatoolkit
!nvidia-smi

!nvcc -o baseline_matmul baseline_matmul.cu
!./baseline_matmul

## 2. Tiling & Shared Memory

That was slow! For N=1024, each thread read **2048** values from global memory. The total global memory traffic was massive.

**Observation:** When computing a `C` tile, threads in a block access the *same* rows of `A` and columns of `B` repeatedly. We are re-fetching the same data from slow DRAM over and over.

**The Solution: Tiling**

1.  **Shared Memory:** Each GPU Streaming Multiprocessor (SM) has a small, *very fast* on-chip memory called **Shared Memory**. It's visible to *all threads in that block*.
2.  **Tiling Strategy:** Break the large `A`, `B`, and `C` matrices into small tiles (e.g., 16x16).
3.  **Cooperative Loading:** Have all threads in a block *cooperate* to load one tile of `A` and one tile of `B` from slow **Global Memory** into fast **Shared Memory**. This is done *once* per tile.
4.  **Synchronize:** Use `__syncthreads()` to create a barrier. This forces all threads to *wait* until the shared memory is fully loaded before proceeding.
5.  **Compute from Shared:** Have each thread compute its part of the `C` tile, but this time reading *only* from fast Shared Memory.
6.  **Loop:** Repeat for all tiles needed to compute the final `C` element.

This strategy maximizes **data reuse**.



**Visual:** Imagine the 16x16 block of threads. 
1. They all work together to grab a 16x16 tile of A and a 16x16 tile of B from global memory and place them in their shared "scratchpad".
2. They wait for everyone to finish (`__syncthreads()`).
3. They compute a 16x16 piece of C, reading only from the fast scratchpad.
4. They move to the next set of tiles, add the result, and repeat.

## 3. Task: Tiled Kernel Implementation

Below is the code for the tiled kernel. Read the comments carefully, especially around `__shared__` and `__syncthreads()`.

In [ ]:
%%writefile tiled_matmul.cu

#include <stdio.h>
#include <stdlib.h>
// We have removed <helper_timer.h>

#define TILE_DIM 16 // Tile dimension

// Helper function (same as before)
void check(cudaError_t err, const char* msg) {
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA Error: %s: %s\n", msg, cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
}

/* 
 * Tiled Kernel: C[row, col] = A[row, :] * B[:, col]
 * Optimized using shared memory.
 */
__global__ void matmul_tiled(float* A, float* B, float* C, int N) {
    
    // 1. Declare Shared Memory tiles
    __shared__ float A_tile[TILE_DIM][TILE_DIM];
    __shared__ float B_tile[TILE_DIM][TILE_DIM];

    // 2. Identify thread position
    int tx = threadIdx.x; // Thread's col within the tile
    int ty = threadIdx.y; // Thread's row within the tile
    int col = blockIdx.x * TILE_DIM + tx; // Global col index
    int row = blockIdx.y * TILE_DIM + ty; // Global row index

    // 3. Loop over tiles to compute one C element
    float val = 0.0f;
    // Robust loop boundary for non-perfect multiples
    for (int tile = 0; tile < (N + TILE_DIM - 1) / TILE_DIM; tile++) {
        
        // 4. Load data from Global to Shared Memory
        int A_col = tile * TILE_DIM + tx;
        int B_row = tile * TILE_DIM + ty;

        if (row < N && A_col < N) {
            A_tile[ty][tx] = A[row * N + A_col];
        } else {
            A_tile[ty][tx] = 0.0f; // Padding
        }
        
        if (B_row < N && col < N) {
            B_tile[ty][tx] = B[B_row * N + col];
        } else {
            B_tile[ty][tx] = 0.0f; // Padding
        }

        // 5. SYNCHRONIZE!
        __syncthreads();

        // 6. Compute using fast Shared Memory
        for (int k = 0; k < TILE_DIM; k++) {
            val += A_tile[ty][k] * B_tile[k][tx];
        }
        
        // 7. SYNCHRONIZE AGAIN!
        __syncthreads();
    }

    // 8. Write the final result to Global Memory
    if (row < N && col < N) {
        C[row * N + col] = val;
    }
}

int main() {
    int N = 1024;
    size_t size = N * N * sizeof(float);
    printf("[Tiled Kernel] Matrix size: %d x %d\n", N, N);

    float* h_A = (float*)malloc(size);
    float* h_B = (float*)malloc(size);
    float* h_C = (float*)malloc(size);
    for (int i = 0; i < N * N; i++) {
        h_A[i] = 1.0f;
        h_B[i] = 2.0f;
    }

    float *d_A, *d_B, *d_C;
    check(cudaMalloc(&d_A, size), "cudaMalloc A");
    check(cudaMalloc(&d_B, size), "cudaMalloc B");
    check(cudaMalloc(&d_C, size), "cudaMalloc C");

    check(cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice), "Memcpy A");
    check(cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice), "Memcpy B");

    // --- START: CUDA Event Timing --- 
    cudaEvent_t start, stop;
    check(cudaEventCreate(&start), "EventCreate start");
    check(cudaEventCreate(&stop), "EventCreate stop");
    // --- END: CUDA Event Timing --- 

    // Block and Grid dimensions are based on TILE_DIM
    dim3 threadsPerBlock(TILE_DIM, TILE_DIM);
    dim3 blocksPerGrid((N + TILE_DIM - 1) / TILE_DIM, (N + TILE_DIM - 1) / TILE_DIM);

    check(cudaEventRecord(start), "EventRecord start");
    matmul_tiled<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);
    check(cudaEventRecord(stop), "EventRecord stop");

    check(cudaEventSynchronize(stop), "EventSynchronize");

    float time_ms = 0;
    check(cudaEventElapsedTime(&time_ms, start, stop), "EventElapsedTime");
    printf("TILED KERNEL TIME: %.3f ms\n", time_ms);

    // Free memory
    check(cudaEventDestroy(start), "EventDestroy start");
    check(cudaEventDestroy(stop), "EventDestroy stop");
    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);

    return 0;
}


In [ ]:
!nvcc -o tiled_gpu tiled_matmul.cu 
!./tiled_gpu

## 4. Performance Comparison

Look at the times! The tiled kernel should be *significantly* faster.

> **REFLECTION**
> 
> * What was your Global Kernel time?
> * What was your Tiled Kernel time?
> * What is the **speedup** (Global Time / Tiled Time)?
> 
> You just saved a huge amount of time by reducing global memory traffic and maximizing data reuse in fast shared memory.



## 5. Synchronization Demo: What if we forget `__syncthreads()`?

This is one of the most common and dangerous bugs in CUDA programming. What happens if we remove the `__syncthreads()` call?

**Hypothesis:** If we remove the sync, threads will enter the compute loop (Step 6) *before* other threads have finished loading the data (Step 4). They will compute using *garbage data* or *stale data* from a previous tile. This is called a **race condition**.

Let's try it. We'll add a verification step to `main()` to check the result.

In [ ]:
%%writefile broken_matmul.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h> // For fabs()

#define TILE_DIM 16

void check(cudaError_t err, const char* msg) {
    if (err != cudaSuccess) {
        fprintf(stderr, "CUDA Error: %s: %s\n", msg, cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }
}

__global__ void matmul_broken(float* A, float* B, float* C, int N) {
    __shared__ float A_tile[TILE_DIM][TILE_DIM];
    __shared__ float B_tile[TILE_DIM][TILE_DIM];

    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int col = blockIdx.x * TILE_DIM + tx;
    int row = blockIdx.y * TILE_DIM + ty;

    float val = 0.0f;
    // Corrected loop boundary
    for (int tile = 0; tile < (N + TILE_DIM - 1) / TILE_DIM; tile++) {
        
        // Load data from Global to Shared Memory
        int A_col = tile * TILE_DIM + tx;
        int B_row = tile * TILE_DIM + ty;

        if (row < N && A_col < N) {
            A_tile[ty][tx] = A[row * N + A_col];
        } else {
            A_tile[ty][tx] = 0.0f;
        }
        
        if (B_row < N && col < N) {
            B_tile[ty][tx] = B[B_row * N + col];
        } else {
            B_tile[ty][tx] = 0.0f;
        }

        // 5. SYNCHRONIZE! - WE COMMENTED THIS OUT!
        // __syncthreads();

        // Compute using (garbage) Shared Memory
        for (int k = 0; k < TILE_DIM; k++) {
            val += A_tile[ty][k] * B_tile[k][tx];
        }
        
        // 7. AND WE COMMENTED OUT THIS ONE TOO!
        // __syncthreads();
    }

    if (row < N && col < N) {
        C[row * N + col] = val;
    }
}

// Main function now has verification
int main() {
    int N = 1024;
    size_t size = N * N * sizeof(float);
    printf("[Broken Kernel] Matrix size: %d x %d\n", N, N);

    float* h_A = (float*)malloc(size);
    float* h_B = (float*)malloc(size);
    float* h_C = (float*)malloc(size);
    for (int i = 0; i < N*N; i++) {
        h_A[i] = 1.0f;
        h_B[i] = 2.0f;
    }

    float *d_A, *d_B, *d_C;
    check(cudaMalloc(&d_A, size), "cudaMalloc A");
    check(cudaMalloc(&d_B, size), "cudaMalloc B");
    check(cudaMalloc(&d_C, size), "cudaMalloc C");
    check(cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice), "Memcpy A");
    check(cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice), "Memcpy B");

    dim3 threadsPerBlock(TILE_DIM, TILE_DIM);
    dim3 blocksPerGrid((N + TILE_DIM - 1) / TILE_DIM, (N + TILE_DIM - 1) / TILE_DIM);

    printf("Launching BROKEN kernel...\n");
    matmul_broken<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, N);
    check(cudaDeviceSynchronize(), "Kernel sync");

    // Copy result back and VERIFY
    check(cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost), "Memcpy C");

    // A=1, B=2. C[i] should be sum(1.0 * 2.0) N times = 2.0 * N
    float expected = 2.0f * N;
    bool success = true;
    for (int i = 0; i < N * N; i++) {
        if (fabs(h_C[i] - expected) > 1e-3) {
            printf("VERIFICATION FAILED at index %d! Got %.2f, expected %.2f\n", 
                   i, h_C[i], expected);
            success = false;
            break;
        }
    }
    if (success) {
        printf("VERIFICATION PASSED! (This is unlikely...)\n");
    }

    cudaFree(d_A); cudaFree(d_B); cudaFree(d_C);
    free(h_A); free(h_B); free(h_C);
    return 0;
}


In [ ]:
!nvcc -o broken_matmul broken_matmul.cu

!./broken_matmul

> 🔹 **INFO: VERIFICATION FAILED!**
> 
> You should see a "VERIFICATION FAILED" message. The output values are garbage. This demonstrates that `__syncthreads()` is not optional—it is **absolutely essential** for correctness when threads in a block share data.

## 6. Recap & Next Steps

> **Key Takeaways**
> 
> * **Global Memory is slow.** Accessing it is a key performance bottleneck.
> * **Shared Memory is fast.** It's a programmable, on-chip cache for a thread block.
> * **Tiling** is a strategy to load data from global to shared memory, compute, and maximize data reuse.
> * **`__syncthreads()`** is a barrier that forces all threads in a block to wait. It is **critical** for correctness when using shared memory to prevent race conditions.

You now have the fundamental building blocks for writing high-performance CUDA code!

### 📚 Resources & Further Reading

* **NERSC GPU Docs:** [NERSC Perlmutter GPU Reference](https://docs.nersc.gov/systems/perlmutter/architecture/#gpus)
* **NVIDIA CUDA Best Practices:** [CUDA C++ Best Practices Guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html) (See the "Shared Memory" section)
* **NERSC Slack & Training:** Ask questions in the `#nersc-users` Slack channel!